<a href="https://colab.research.google.com/github/jiin124/Deep_learning/blob/main/%EC%8B%AC%EC%B8%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 심층 신경망



In [2]:
from tensorflow import keras

(train_input,train_target),(test_input,test_target)=keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


이미지의 픽셀값을 0~255 범위에서 0~1사이로 변화하고 28*28 크기의 2차원 배열을 784 크기의 1차원 배열로 펼친다. 

In [3]:
from sklearn.model_selection import train_test_split

train_scaled=train_input/255.0
train_scaled=train_scaled.reshape(-1,28*28)
train_scaled,val_scaled,train_target,val_target=train_test_split(train_scaled,train_target,test_size=0.2,random_state=42)

이제 인공 신경망 모델에 층을 2개 추가해보겠다. 

입력층과 출력층 사이에 있는 모든 층을 은닉층이라고 한다. 
은닉층에는 활성화 함수가 표시되어있는데 활성화 함수는 신경망 층의 선형 방정식의 계산 값에 사용하는 값이다. 이전에 출력층에 적용했던 소프트맥스 함수도 활성화함수이다. 출려긍에 적용하는 활성화 함수는 종류가 제한되어있다. 이진분류일경우 시그모이드 함수를 사용하고 다중 분류일 경우 소프트 맥스 함수를 사용한다. 이에 비해 은닉층의 활성화 함수는 비교적 자유롭다. 대표적으로 시그모이드 함수와 렐루 함수를 사용한다.

- 회귀를 위한 신경망의 출력층에서는 어떤 활성화 함수를 사용할까?
분류문제는 클래스에 대한 확률을 출력하기 위해 활성화 함수를 사용한다. 회귀의 출력은 임의의 어떤 숫자이므로 활성화 함수를 적용할 필요가 없다. 출력층의 선형 방정식의 계산을 그대로 출력한다. 이렇게 하려면 Dense 층의 activation 매개변수에 아무런 값을 지정하지 않는다. 

- 은닉층에 활성화 함수를 사용하는 이유

은닉층에서 선형적인 산술 계산만 수행한다면 수행 역할이 없다. 선형 계산을 적당하게 비선형적으로 비틀어주어야 한다. 그래야 다음 층의 꼐산과 단순히 합쳐지지 않고 나름의 역할을 할 수 있다. 


In [4]:
dense1=keras.layers.Dense(100,activation='sigmoid',input_shape=(784,))
dense2=keras.layers.Dense(10,activation='softmax')

dense1이 은닉층이고 100개의 뉴런을 가진 밀집층이다 활성화 함수를 시그모이드로 지정했고 input_shape에서 입력의 크기를 (784,)로 지정했다. 은닉층의 뉴런 개수를 정하는데는 특별한 기준이 없다. 몇 개의 뉴런을 두어야 할지 판단하기 위해서는 상당한 경험이 필요하다. 


여기서 한가지 제약 사항이 있다면 적어도 출력층의 뉴런보다는 많게 만들어야 한다. 클래스 10개에 대한 확률을 예측해야하는데 이전 은닉층이 10개보다 적다면 부족한 정보가 전달될 것이다. 

그다음은 dense2는 출력층이다. 10개의 클래스를 분류하므로, 10개의 뉴런을 두었고 활성화 함수는 소프트 맥스 함수로 지정했다. 

## 심층 신경망 만들기

앞에서 만든 dense1,dense2 객체를 Sequential 클래스에 추가해 심층 신경망(Deep neural network DNN)을 만들어보겠다. 

In [5]:
model=keras.Sequential([dense1,dense2])

Sequential 클래스의 객체를 만들 때 여러개의 층을 추가하려면 리스트로 만들어 전달해야한다. 여기서 주의할 것은 출력층을 가장 마지막에 두어야 한다는 것. 

인공 신경망의 강력한 성능은 바로 이렇게 층을 추가해 입력 데이터에 대해 연속적인 학습을 진행하는 능력에서 나온다. 

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


맨 첫줄에 모델의 이름이 나온다 그다음 이 모델에 들어있는 층이 순서대로 나열된다 이 순서는 맨 처음 추가한 은닉층에서 출력층의 순서대로 나열된다 .층마다 층 이름, 클래스, 출력크기, 모델 파라미터 개수가 출련된다 층을 만들 떄 name 매개변수로 이름을 지정할 수 있다. 층 이름을 지정하지 않으면 케라스가 자동으로 dense라고 이름을 붙인다. 

출력의 크기를 보면 (None, 100)이다. 첫번째 차언은 샘플의 개수를 나타낸다. 샘플 개수가 아직 정의되어있지 않기 때문에 None이다. 왤까? 케라스 모델의 fit()메서드에 훈련데이터를 주입하면 이 데이터를 한번에 모두 사용하지 않고 잘게 나누어 여러번에 걸쳐 경사 하강법 단계를 수행한다. 바로 미니배치 경사 하강법을 사용하는 것이다. 

케라스의 기본 미니 배치 크기는 32개이다. 이 값은 fit()메서드에서 batch_size 매개변수로 바꿀 수 있다. 따라서 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정한다. 이렇게 신경망 층에 입력되거나 출력되는 배열의 첫번째 차원을 배치 차원이라고 부른다. 

두번째 100은 쉽다. 은닉층의 뉴런개수를 100개로 두었으니 100개의 출력이 나온다. 즉 샘플마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축되었다. 

마지막으로 모델 파라미터 개수가 출력된다. 이 층은 Dense 층이므로 입력 픽셀 784개와 100개의 모든 조합에 대한 가중치가 있다. 그리고 뉴런마다 1개의 절편이 있따.
784*100+100=78500

두번째 층의 출력크기는 (None,10)이다. 배치차원은 동일하게 None이고 출력 뉴런 개수가 10개이기 때문이다. 이 층의 모델 파라미터 개수는
10*100+10=1010

summary() 메서드의 마지막에는 총 모델 파라미터 개수와 훈련되는 파라미터 개수가 동일하게 79510개로 나온다. 은닉층과 출력층의 파라미터 개수를 합친 값이다. 

## 층을 추가하는 다른 방법

sequential 클래스에 층을 추가하는 다른 방법을 알아보자, 앞에서는 Dense 클래스의 객체 dense1,dense2를 만들어 sequential 클래스에 전달했다. 이 두 객체를 따로 저장해 쓸일이 없기 때문에 다음 처럼 sequential 클래스의 생성자 안에서 바로 dense 클래스의 객체를 만드는 경우가 많다.

In [7]:
model=keras.Sequential([
                        keras.layers.Dense(100,activation='sigmoid',input_shape=(784,),
                                           name='hidden'),
                        keras.layers.Dense(10,activation='softmax',name='output')
],name='패션 MNIST모델')

이렇게 작업하면 추가되는 층을 한눈에 쉽게 알아보는 장점이 있다 이전과는 달리 이번에는 sequential 클래스의 name 매개변수로 모델의 이름을 정했다. 또 Dense 층의 name 매개변수에 층의 이름을 'hidden'과 'output'으로 각각 지정했다. 

In [8]:
model.summary()

Model: "패션 MNIST모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


여러 모델과 많은 층을 사용할 때 name 매개변수를 사용하면 구분하기 쉽다. 이 방법이 편리하지만 아주 많은 층을 추가하려면 sequential 클래스 생성자가 매우 길어진다. 또 조건에 따라 층을 추가할 수도 없다. Sequential 클래스에서 층을 추가할 떄 가장 널리 사용하는 방법은 add() 메서드이다. 

In [10]:
model=keras.Sequential()
model.add(keras.layers.Dense(100,activation='sigmoid',input_shape=(784,)))
model.add(keras.layers.Dense(10,activation='softmax'))

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


이제 모델을 훈련해보자 compile() 메서드의 설정은 1절에서 했던 것과 동일하다. 5번의 에포크동안 훈련해보자. 

In [13]:
model.compile(loss='sparse_categorical_crossentropy',metrics='accuracy')
model.fit(train_scaled,train_target,epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5638 - accuracy: 0.8097
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4063 - accuracy: 0.8538
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3724 - accuracy: 0.8668
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3493 - accuracy: 0.8748
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3314 - accuracy: 0.8800


추가된 층이 성능을 향상시킨 것을 알 수 있다. 

## 렐루 함수

초창기 인공 신경망의 은닉층에 많이 사용된 활성화 함수는 시그모이드 함수이다. 하지만 이 함수에는 단점이 있다. 이 함수의 오른쪽과 왼쪽 끝으로 갈수로 그래프가 누워있기 떄문에 올바른 출력을 만드는데 신속하게 대응하지 못한다. 

In [14]:
model=keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(10,activation='softmax'))

첫번째 Dense층에 있던 input_shape 매개변수를 flatten 층으로 옮겼다. 또 첫번째 dense 층의 활성화 함수를 relu로 바꾸었다. 하지만 이 신경망을 깊이가 3인 신경망이라고 부르진 않는다.flatten 클래스는 학습하는 층이 아니기 때문이다. 

In [15]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 100)               78500     
                                                                 
 dense_7 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


flatten클래스에 포함된 모델 파라미터는 0개이다. 케라스의 flatten 층을 신경망 모델에 추가하면 입력값의 차원을 짐작할 수 있는 것이 또하나의 장점이다. 임력데이터에 대한 전처리과정을 가능한 모델에 포함시키는 것이 케라스 aPi의 철학 중 하나이다. 

In [16]:
(train_input,train_target),(test_input,test_target)=keras.datasets.fashion_mnist.load_data()
train_scaled=train_input/255.0
train_scaled,val_scaled,train_target,val_target=train_test_split(train_scaled,train_target,test_size=0.2,random_state=42)

In [17]:
model.compile(loss='sparse_categorical_crossentropy',metrics='accuracy')
model.fit(train_scaled,train_target,epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5354 - accuracy: 0.8108
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3969 - accuracy: 0.8575
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3580 - accuracy: 0.8709
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3357 - accuracy: 0.8791
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3245 - accuracy: 0.8845


음, 크지는 않지만 렐루 함수의 효과를 본것 같다. 이제 검증 세트에서의 성능도 확인 ㄱ

In [18]:
model.evaluate(val_scaled,val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3703 - accuracy: 0.8745


[0.37034687399864197, 0.8744999766349792]

In [21]:
model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy',
              metrics='accuracy')

In [23]:
sgd=keras.optimizers.SGD()
model.compile(optimizer=sgd,loss='sparse_categorical_crossentropy',metrics='accuracy')

In [24]:
sgd=keras.optimizers.SGD(learning_rate=0.1)

In [25]:
adagrad=keras.optimizers.Adagrad()
model.compile(optimizer=adagrad,loss='sparse_categorical_crossentropy',
              metrics='accuracy')

In [26]:
rmsprop=keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop,loss='sparse_cagegorical_crossentropy',metrics='accuracy')

In [27]:
model=keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(10,activation='softmax'))

In [28]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics='accuracy')
model.fit(train_scaled,train_target,epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5207 - accuracy: 0.8179
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3922 - accuracy: 0.8597
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3500 - accuracy: 0.8722
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3245 - accuracy: 0.8815
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3041 - accuracy: 0.8884


In [29]:
model.evaluate(val_scaled,val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3301 - accuracy: 0.8823


[0.3301205337047577, 0.8822500109672546]